<a href="https://colab.research.google.com/github/karanpraharaj/Practice/blob/master/wordembedsprac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **N-GRAM LANGUAGE MODEL** 

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [0]:
word_to_ix = {"hello":0, "world":1}
embeds = nn.Embedding(2,5)  #2 words in vocab, 5 dimensional embedding
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)

In [20]:
embeds(lookup_tensor)

tensor([[-0.9081,  0.5423,  0.1103, -2.2590,  0.6067]],
       grad_fn=<EmbeddingBackward>)

In [0]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [0]:
trigram = [([test_sentence[i], test_sentence[i+1]], test_sentence[i+2]) for i in range(len(test_sentence)-2)]

In [0]:
vocab = set(test_sentence)
vocab = sorted(vocab)

In [0]:
word_to_ix = {w : i for i,w in enumerate(sorted(vocab))}

In [0]:
class ngramlangmodel(nn.Module):
  
  def __init__(self,vocab_size,embed_size, context_size):
    super(ngramlangmodel,self).__init__()
    self.embedding = nn.Embedding(vocab_size,embed_size)
    self.linear1 = nn.Linear(context_size*embed_size, 128)
    self.linear2 = nn.Linear(128, vocab_size)
  
  def forward(self,inputs):
    embeds = self.embedding(inputs).view(1,-1)
    out = self.linear1(embeds)
    out = F.relu(out)
    out = self.linear2(out)
    log_probs = F.log_softmax(out,dim=1)
    
    return log_probs
  
losses = []
lossfn = nn.NLLLoss()
model = ngramlangmodel(len(vocab),EMBEDDING_DIM,CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)
    

In [71]:
for epoch in range(10):
  total_loss = 0
  for context, target in trigram:
    # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words into integer indices and wrap them in tensors)
    cont2indx = torch.tensor([word_to_ix[w] for w in context], dtype = torch.long)
    targetind = torch.tensor([word_to_ix[target]],dtype = torch.long)
    
    # Step 2. Recall that torch *accumulates* gradients. Before passing in a new instance, you need to zero out the gradients from the old instance
    model.zero_grad()
    
    # Step 3. Run the forward pass, getting log probabilities over next words
    logprobs = model(cont2indx)
    
    # Step 4. Compute your loss function. (Again, Torch wants the target word wrapped in a tensor)
    loss = lossfn(logprobs,targetind)
    
    # Step 5. Do the backward pass and update the gradient    
    loss.backward()
    optimizer.step()
    
    total_loss += loss.item()
    
  losses.append(total_loss)
  print("Loss" , epoch, total_loss)
  
  
  
    
    

Loss 0 23.919101238250732
Loss 1 23.743408679962158
Loss 2 23.570165157318115
Loss 3 23.399426460266113
Loss 4 23.23105001449585
Loss 5 23.06493330001831
Loss 6 22.900967597961426
Loss 7 22.739248752593994
Loss 8 22.57994794845581
Loss 9 22.422611713409424


# **Continuous Bag of Words**


In [0]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()


In [0]:
vocab = set(raw_text)
vocab = sorted(vocab)
lenvocab = len(vocab)
word_to_ix = {word: i for i, word in enumerate(vocab)}


In [0]:
context = [([raw_text[i-2], raw_text[i-1], raw_text[i+1], raw_text[i+2]]) for i in range(2, len(raw_text)-2)]
target = [raw_text[i] for i in range(2,len(raw_text)-2)]
data = [(context[i],target[i]) for i in range(len(context)) ]

In [0]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

In [105]:
make_context_vector(context[1], word_to_ix)

tensor([ 9,  7, 41, 43])

[(['We', 'are', 'to', 'study'], 'about'),
 (['are', 'about', 'study', 'the'], 'to'),
 (['about', 'to', 'the', 'idea'], 'study'),
 (['to', 'study', 'idea', 'of'], 'the'),
 (['study', 'the', 'of', 'a'], 'idea'),
 (['the', 'idea', 'a', 'computational'], 'of'),
 (['idea', 'of', 'computational', 'process.'], 'a'),
 (['of', 'a', 'process.', 'Computational'], 'computational'),
 (['a', 'computational', 'Computational', 'processes'], 'process.'),
 (['computational', 'process.', 'processes', 'are'], 'Computational'),
 (['process.', 'Computational', 'are', 'abstract'], 'processes'),
 (['Computational', 'processes', 'abstract', 'beings'], 'are'),
 (['processes', 'are', 'beings', 'that'], 'abstract'),
 (['are', 'abstract', 'that', 'inhabit'], 'beings'),
 (['abstract', 'beings', 'inhabit', 'computers.'], 'that'),
 (['beings', 'that', 'computers.', 'As'], 'inhabit'),
 (['that', 'inhabit', 'As', 'they'], 'computers.'),
 (['inhabit', 'computers.', 'they', 'evolve,'], 'As'),
 (['computers.', 'As', 'evol